In [1]:
from sys import platform
if platform == 'linux' or platform == 'linux2':
    datapath = '/home/alin/MyLearning/Kaggle/Sberbank/data/'
else:
    datapath = 'C:/Users/alin/Documents/SelfStudy/MyLearning/Kaggle/Sberbank/data/'
print datapath

/home/alin/MyLearning/Kaggle/Sberbank/data/


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [3]:
import xgboost as xgb

In [4]:
#from xgboost.sklearn import XGBClassifier

In [5]:
#%matplotlib inline

In [4]:
# From here: https://www.kaggle.com/robertoruiz/sberbank-russian-housing-market/dealing-with-multicollinearity/notebook
macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

In [5]:
%qtconsole

In [15]:
train_df = pd.read_csv(datapath + 'train.csv', parse_dates=['timestamp'])
test_df = pd.read_csv(datapath + 'test.csv', parse_dates = ['timestamp'])
macro_df = pd.read_csv(datapath + 'macro.csv', parse_dates=['timestamp'], usecols=['timestamp'] + macro_cols)


In [34]:
ylog_train_all = np.log1p(train_df['price_doc'].values)


KeyError: 'price_doc'

In [17]:
train_df.drop(['id', 'price_doc'], axis = 1, inplace = True)
tid = test_df.id.copy()
test_df.drop(['id'], axis = 1, inplace = True)

In [18]:
all_df = pd.merge(train_df, macro_df, on = 'timestamp', how = 'left')
all_test = pd.merge(test_df, macro_df, on = 'timestamp', how = 'left')

In [19]:
month_year = (all_df.timestamp.dt.month + all_df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
all_df['month_year_cnt'] = month_year.map(month_year_cnt_map)


test_month_year = (all_test.timestamp.dt.month + all_test.timestamp.dt.year * 100)
test_month_year_cnt_map = test_month_year.value_counts().to_dict()
all_test['month_year_cnt'] = test_month_year.map(test_month_year_cnt_map)

In [20]:
week_year = (all_df.timestamp.dt.weekofyear + all_df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
all_df['week_year_cnt'] = week_year.map(week_year_cnt_map)

test_week_year = (all_test.timestamp.dt.weekofyear + all_test.timestamp.dt.year * 100)
test_week_year_cnt_map = test_week_year.value_counts().to_dict()
all_test['week_year_cnt'] = test_week_year.map(test_week_year_cnt_map)


In [21]:
all_df['month'] = all_df.timestamp.dt.month
all_df['dow'] = all_df.timestamp.dt.dayofweek

all_test['month'] = all_test.timestamp.dt.month
all_test['dow'] = all_test.timestamp.dt.dayofweek


In [22]:
all_df['rel_floor'] = all_df['floor'] / all_df['max_floor'].astype(float)
all_test['rel_floor'] = all_test['floor'] / all_test['max_floor'].astype(float)

In [23]:
all_df['rel_kitch_sq'] = all_df['kitch_sq'] / all_df['full_sq'].astype(float)
all_test['rel_kitch_sq'] = all_test['kitch_sq'] / all_test['full_sq'].astype(float)

In [24]:
all_df.drop(['timestamp'], axis = 1, inplace = True)
all_test.drop(['timestamp'], axis = 1, inplace = True)

In [27]:
num_df = all_df.select_dtypes(include = [np.number])
cat_df = all_df.select_dtypes(exclude = [np.number]).copy()

test_num_df = all_test.select_dtypes(include = [np.number])
test_cat_df = all_test.select_dtypes(exclude = [np.number]).copy()

In [28]:
for c in cat_df:
    cat_df[c] = pd.factorize(cat_df[c])[0]
    
for c in test_cat_df:
    test_cat_df[c] = pd.factorize(test_cat_df[c])[0]

In [29]:
df_values = pd.concat([num_df, cat_df], axis = 1)

test_values = pd.concat([test_num_df, test_cat_df], axis = 1)

In [30]:
X_all = df_values.values
X_test = test_values.values

In [31]:
df_columns = df_values.columns

In [37]:
dtrain = xgb.DMatrix(X_all, ylog_train_all, feature_names=df_columns)

In [38]:
dtest = xgb.DMatrix(X_test, feature_names=df_columns)

In [39]:
#def rmsle(lp, la):
#    return np.sqrt(np.mean(np.square(lp - la)))

In [40]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 5,
     'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [41]:
cvresult = xgb.cv(xgb_params, dtrain, num_boost_round=1000, nfold=5,
            metrics='rmse', early_stopping_rounds=50, verbose_eval=True)

[0]	train-rmse:13.6116+0.00134418	test-rmse:13.6118+0.00576962
[1]	train-rmse:12.253+0.00114206	test-rmse:12.2533+0.00581466
[2]	train-rmse:11.0303+0.00107428	test-rmse:11.0303+0.00577657
[3]	train-rmse:9.93013+0.000988969	test-rmse:9.93004+0.00585588
[4]	train-rmse:8.94001+0.000914724	test-rmse:8.93994+0.00595607
[5]	train-rmse:8.04933+0.000802956	test-rmse:8.04926+0.00581896
[6]	train-rmse:7.24789+0.000862564	test-rmse:7.24781+0.00571481
[7]	train-rmse:6.52702+0.000749533	test-rmse:6.52706+0.00574875
[8]	train-rmse:5.87839+0.00067804	test-rmse:5.87849+0.00547783
[9]	train-rmse:5.29509+0.000606661	test-rmse:5.29517+0.00552826
[10]	train-rmse:4.77066+0.000573571	test-rmse:4.77086+0.00555212
[11]	train-rmse:4.29923+0.000567435	test-rmse:4.29949+0.00540612
[12]	train-rmse:3.87522+0.000451203	test-rmse:3.87554+0.00540791
[13]	train-rmse:3.49423+0.00043294	test-rmse:3.49494+0.00547622
[14]	train-rmse:3.15182+0.000385634	test-rmse:3.15283+0.00529388
[15]	train-rmse:2.84432+0.000365403	test-

In [42]:
num_rounds = 160

In [43]:
model0 = xgb.train(xgb_params, dtrain, num_boost_round = num_rounds, evals = [(dtrain, 'train')], verbose_eval = 20)

[0]	train-rmse:13.6116
[20]	train-rmse:1.72215
[40]	train-rmse:0.491769
[60]	train-rmse:0.435083
[80]	train-rmse:0.424119
[100]	train-rmse:0.41431
[120]	train-rmse:0.407212
[140]	train-rmse:0.399462
[159]	train-rmse:0.392983


In [44]:
y_pred1 = model0.predict(dtest)


In [47]:
price1 = np.exp(y_pred1) - 1
output1 = pd.DataFrame({'id':tid, 'price_doc': price1})
output1.to_csv('/home/alin/MyLearning/Kaggle/Sberbank/data/submission3.csv', index = False)

In [48]:
import lightgbm as lgb

In [49]:
lgb_train = lgb.Dataset(X_all, ylog_train_all)
lgb_test = lgb.Dataset(X_test)

In [50]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': {'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
    }


In [51]:
cv_lgb = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold=5, early_stopping_rounds = 50, 
                verbose_eval = True)

[1]	cv_agg's rmse: 0.594002 + 0.00715451
[2]	cv_agg's rmse: 0.584334 + 0.00706042
[3]	cv_agg's rmse: 0.575362 + 0.00695474
[4]	cv_agg's rmse: 0.567243 + 0.00689153
[5]	cv_agg's rmse: 0.559678 + 0.00686926
[6]	cv_agg's rmse: 0.552805 + 0.00678876
[7]	cv_agg's rmse: 0.546433 + 0.00669059
[8]	cv_agg's rmse: 0.540624 + 0.00662611
[9]	cv_agg's rmse: 0.535171 + 0.0065282
[10]	cv_agg's rmse: 0.530166 + 0.00633255
[11]	cv_agg's rmse: 0.525527 + 0.00625786
[12]	cv_agg's rmse: 0.521281 + 0.00612695
[13]	cv_agg's rmse: 0.517397 + 0.00602595
[14]	cv_agg's rmse: 0.513871 + 0.00590885
[15]	cv_agg's rmse: 0.510513 + 0.00589572
[16]	cv_agg's rmse: 0.507465 + 0.00577046
[17]	cv_agg's rmse: 0.504614 + 0.00574196
[18]	cv_agg's rmse: 0.502011 + 0.00572196
[19]	cv_agg's rmse: 0.499657 + 0.00557755
[20]	cv_agg's rmse: 0.497557 + 0.00549843
[21]	cv_agg's rmse: 0.495476 + 0.00542067
[22]	cv_agg's rmse: 0.493567 + 0.00535888
[23]	cv_agg's rmse: 0.491796 + 0.00526665
[24]	cv_agg's rmse: 0.490138 + 0.00525892
[2

In [52]:
gbm = lgb.train(params,lgb_train,num_boost_round=227)

In [53]:
y_pred2 = gbm.predict(X_test)
price2 = np.exp(y_pred2) - 1
output2 = pd.DataFrame({'id':tid, 'price_doc': price2})
output2.to_csv('/home/alin/MyLearning/Kaggle/Sberbank/data/submission4.csv', index = False)